# Generic

This is mostly copied from the previous notebook

In [1]:
import math
import base64
import binascii
import numpy as np
import matplotlib.pyplot as plt

def to_bytes(d, format):
    if isinstance(d, (bytes, bytearray)):
        return d
    elif format == 'hex':
        return bytes(bytearray.fromhex(d))
    elif format == 'base64':
        return base64.b64decode(d)
    elif format == 'str' or format == 'bytes':
        return d.encode()
    elif format == 'int':
        return to_bytes(hex(d)[2:], 'hex')

def bytes_to(b, format):
    if not isinstance(b, (bytes, bytearray)):
        return b
    elif format == 'hex':
        return binascii.hexlify(b).decode()
    elif format == 'base64':
        return base64.b64encode(b).decode()
    elif format == 'str':
        return b.decode()
    elif format == 'bytes':
        return b
    elif format == 'int':
        return int(bytes_to(b, 'hex')[2:], 16)
    
def draw_bytestring(bs):
    """Represent a byte string as a matrix of color
    """
    tmp = bs + bytes(math.ceil(math.sqrt(len(bs)))**2 - len(bs))
    tmp = [int(b) for b in tmp]
    m = [[tmp[i:i+int(math.sqrt(len(tmp)))] for i in range(0, len(tmp), int(math.sqrt(len(tmp))))]]
    matrix = np.matrix(m[0])
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.set_aspect('equal')
    plt.imshow(matrix, interpolation='nearest', cmap=plt.cm.binary)
    plt.colorbar()
    plt.show()

## Helpers

This is mostly copied from the previous notebook

# Cryptopals notebook - part 3

## Set 4

### Exercice 25